# Preços de Medicamentos - Consumidor
### Nesse projeto irei criar uma API para saber de fato de quanto foi o aumento no preço dos médicamento nesse último reajuste no brasil.

<img src="https://i.imgur.com/W7DMmnl.jpeg" />

Os dados representam a lista de preços de Medicamentos , contemplando o preço Fábrica, ou preço **fabricante (PF)**, que é o preço máximo que pode ser praticado pelas empresas produtoras ou importadoras do produto e pelas empresas distribuidoras. O PF indica o preço máximo permitido para venda a farmácias e drogarias e o **Preço Máximo ao Consumidor (PMC)** indica o preço teto de venda ao consumidor.

Fonte dos dados: https://dados.anvisa.gov.br/dados

### Agência Nacional de Vigilância Sanitária - ANVISA

Criada pela Lei nº 9.782, de 26 de janeiro 1999, a Agência Nacional de Vigilância Sanitária (Anvisa) é uma autarquia sob regime especial, que tem sede e foro no Distrito Federal, e está presente em todo o território nacional por meio das coordenações de portos, aeroportos, fronteiras e recintos alfandegados.

Tem por finalidade institucional promover a proteção da saúde da população, por intermédio do controle sanitário da produção e consumo de produtos e serviços submetidos à vigilância sanitária, inclusive dos ambientes, dos processos, dos insumos e das tecnologias a eles relacionados, bem como o controle de portos, aeroportos, fronteiras e recintos alfandegados.

- Data da criação do banco de dados: 31 de maio de 2017
- Penúltima atualização: 13 de outubro de 2020 para o DataFrame anterior
- Ultima atualização: 08 de fevereiro de 2021 para o DataFrame reajuste

In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import folium
%matplotlib inline



import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 999)

In [2]:
import mysql.connector
import pandas as a

conn = mysql.connector.connect(host="localhost", user="root", passwd="1234", db="preco_medicamentos_brasil")
anterior = a.read_sql_query("SELECT * FROM TA_PRECO_MEDICAMENTO_10_2020" 
                      ,conn)


reajuste = a.read_sql_query("SELECT * FROM TA_PRECO_MEDICAMENTO_03_2021" 
                      ,conn)

In [3]:
anterior.head()

,SUBSTANCIA,CNPJ,LABORATORIO,CODIGO GGREM,REGISTRO,EAN 1,EAN 2,EAN 3,PRODUTO,APRESENTACAO,CLASSE TERAPEUTICA,TIPO DE PRODUTO (STATUS DO PRODUTO),REGIME DE PRECO,PF Sem Impostos,PF 0%,PF 12%,PF 17%,PF 17% ALC,"PF 17,5%","PF 17,5% ALC",PF 18%,PF 18% ALC,PF 20%,PMC 0%,PMC 12%,PMC 17%,PMC 17% ALC,"PMC 17,5%","PMC 17,5% ALC",PMC 18%,PMC 18% ALC,PMC 20%,RESTRICAO HOSPITALAR,CAP,CONFAZ 87,ICMS 0%,ANALISE RECURSAL,LISTA DE CONCESSAO DE CREDITO TRIBUTARIO (PIS/COFINS),COMERCIALIZACAO 2019,TARJA
0,ACETATO DE DEXAMETASONA;CLOTRIMAZOL,18.459.628/0001-15,BAYER S.A.,"5,39E+14","1,71E+12","7,89E+12",-,-,BAYCUTEN N,"10 MG/G + 0,443 MG/G CREM DERM CT BG AL X 40 G",D7B2 - CORTICOESTERÓIDES ASSOCIADOS A ANTIMICO...,Novo,Regulado,"19,74","22,11","25,55","27,32","23,78","27,51","23,93","27,71","27,71","28,51","29,66","34,13","36,42","32,87","36,66","33,08","36,92","33,28","37,95",Năo,Năo,Năo,Năo,,Negativa,Sim,Tarja Vermelha
1,ABATACEPTE,56.998.982/0001-07,BRISTOL-MYERS SQUIBB FARMACĘUTICA LTDA,"5,05E+14","1,02E+12","7,90E+12",-,-,ORENCIA,125 MG/ML SOL INJ CT 1 SER PREENCHIDA,M1C - AGENTES ANTI-REUMÁTICOS ESPECÍFICOS,Biológico,Regulado,"1228,49","1228,49","1396,01","1480,11","1480,11","1489,08","1489,08","1498,16","1498,16","1535,61","1698,32","1929,9","2046,17","2046,17","2058,57","2058,57","2071,12","2071,12","2122,89",Năo,Năo,Năo,Năo,,Positiva,Năo,Tarja Vermelha
2,ABATACEPTE,56.998.982/0001-07,BRISTOL-MYERS SQUIBB FARMACĘUTICA LTDA,"5,05E+14","1,02E+12","7,90E+12",-,-,ORENCIA,125 MG/ML SOL INJ CT 4 SER PREENC VD TRANS + D...,M1C - AGENTES ANTI-REUMÁTICOS ESPECÍFICOS,Biológico,Regulado,"4914,04","4914,04","5584,14","5920,53","5920,53","5956,41","5956,41","5992,73","5992,73","6142,55","6793,37","7719,75","8184,79","8184,79","8234,39","8234,39","8284,6","8284,6","8491,72",Năo,Năo,Sim,Năo,,Positiva,Sim,Tarja Vermelha (*)
3,ABATACEPTE,56.998.982/0001-07,BRISTOL-MYERS SQUIBB FARMACĘUTICA LTDA,"5,05E+14","1,02E+12","7,90E+12",-,-,ORENCIA,250 MG PO LIOF INJ 1 CT 1 FA + SER DESCARTÁVEL,M1C - AGENTES ANTI-REUMÁTICOS ESPECÍFICOS,Biológico,Regulado,"1547,45","1547,45","1758,47","1864,4","1864,4","1875,7","1875,7","1887,13","1887,13","1934,31",,,,,,,,,,Sim,Năo,Năo,Năo,,Positiva,Sim,Tarja Vermelha
4,ABCIXIMABE,51.780.468/0001-87,JANSSEN-CILAG FARMACĘUTICA LTDA,"5,15E+14","1,12E+12","7,90E+12",-,-,REOPRO,2 MG/ML SOL INJ CT FA VD INC X 5 ML,"B1C3 - INIBIDORES DA AGREGAÇĂO PLAQUETÁRIA, AN...",-,Regulado,"2003,06","2003,06","2276,21","2413,32","2413,32","2427,95","2427,95","2442,76","2442,76","2503,83",,,,,,,,,,Sim,Năo,Năo,Năo,,Positiva,Sim,Tarja Vermelha (*)


In [4]:
reajuste.head()

,SUBSTANCIA (REAJUSTE),CNPJ (REAJUSTE),LABORATORIO (REAJUSTE),CODIGO GGREM (REAJUSTE),REGISTRO (REAJUSTE),EAN 1 (REAJUSTE),EAN 2 (REAJUSTE),EAN 3 (REAJUSTE),PRODUTO (REAJUSTE),APRESENTACAO (REAJUSTE),CLASSE TERAPEUTICA (REAJUSTE),TIPO DE PRODUTO (STATUS DO PRODUTO) (REAJUSTE),REGIME DE PRECO (REAJUSTE),PF Sem Impostos (REAJUSTE),PF 0% (REAJUSTE),PF 12% (REAJUSTE),PF 17% (REAJUSTE),PF 17% ALC (REAJUSTE),"PF 17,5% (REAJUSTE)","PF 17,5% ALC (REAJUSTE)",PF 18% (REAJUSTE),PF 18% ALC (REAJUSTE),PF 20% (REAJUSTE),PMC 0% (REAJUSTE),PMC 12% (REAJUSTE),PMC 17% (REAJUSTE),PMC 17% ALC (REAJUSTE),"PMC 17,5% (REAJUSTE)","PMC 17,5% ALC (REAJUSTE)",PMC 18% (REAJUSTE),PMC 18% ALC (REAJUSTE),PMC 20% (REAJUSTE),RESTRICAO HOSPITALAR (REAJUSTE),CAP (REAJUSTE),CONFAZ 87 (REAJUSTE),ICMS 0% (REAJUSTE),ANALISE RECURSAL (REAJUSTE),LISTA DE CONCESSAO DE CREDITO TRIBUTARIO (PIS/COFINS) (REAJUSTE),COMERCIALIZACAO 2019 (REAJUSTE),TARJA (REAJUSTE)
0,ACETATO DE LANREOTIDA,07.718.721/0001-80,BEAUFOUR IPSEN FARMACÊUTICA LTDA,"5,39E+14","1,70E+12","7,90E+12","7,90E+12",-,SOMATULINE AUTOGEL,120 MG SOL INJ LIB PROL CT ENV AL/PLAS SER PRE...,H1C2 - HORMÔNIOS ANTICRESCIMENTO,Novo,Regulado,"2456,35","2456,35","2791,31","2959,46","2959,46","2977,39","2977,39","2995,55","2995,55","3070,44","3395,76","3858,82","4091,28","4091,28","4116,07","4116,07","4141,17","4141,17","4244,7",Não,Sim,Sim,Não,,Positiva,Não,Tarja -(*)
1,ACICLOVIR,45.992.062/0001-65,GERMED FARMACEUTICA LTDA,"5,26E+14","1,06E+12","7,90E+12",-,-,ACICLOVIR,400 MG COM CT BL AL PLAS PVDC OPC X 20,J5B3 - ANTIVIRAIS PARA HERPES,Genérico,Regulado,"57,69","57,69","65,56","69,51","69,51","69,93","69,93","70,35","70,35","72,11","79,75","90,63","96,09","96,09","96,67","96,67","97,25","97,25","99,69",Não,Não,Não,Não,,Positiva,Não,Tarja Vermelha(*)
2,ACICLOVIR,45.992.062/0001-65,GERMED FARMACEUTICA LTDA,"5,26E+14","1,06E+12","7,90E+12",-,-,ACICLOVIR,400 MG COM CT BL AL PLAS PVDC OPC X 30,J5B3 - ANTIVIRAIS PARA HERPES,Genérico,Regulado,"86,54","86,54","98,34","104,27","104,27","104,9","104,9","105,54","105,54","108,18","119,64","135,95","144,15","144,15","145,02","145,02","145,9","145,9","149,55",Não,Não,Não,Não,,Positiva,Não,Tarja Vermelha(*)
3,ACICLOVIR,45.992.062/0001-65,GERMED FARMACEUTICA LTDA,"5,26E+14","1,06E+12","7,90E+12",-,-,ACICLOVIR,400 MG COM CT BL AL PLAS PVDC OPC X 50,J5B3 - ANTIVIRAIS PARA HERPES,Genérico,Regulado,"144,23","144,23","163,9","173,77","173,77","174,82","174,82","175,89","175,89","180,29","199,39","226,58","240,23","240,23","241,68","241,68","243,16","243,16","249,24",Não,Não,Não,Não,,Positiva,Não,Tarja Vermelha(*)
4,ACTAEA RACEMOSA L.,78.950.011/0001-20,HERBARIUM LABORATORIO BOTANICO LTDA,"5,11E+14","1,19E+12","7,90E+12",-,-,CLIFEMIN,160 MG COM REV CT BL AL PLAS PVDC TRANS X 15,G3J - MODULADORES SELETIVOS DO RECEPTOR DE EST...,Fitoterápico,Liberado,"34,1","38,19","44,13","47,19","41,08","47,52","41,33","47,86","41,58","49,24","51,23","58,95","62,9","56,79","63,33","57,14","63,76","57,48","65,54",Não,Não,Não,Não,,Negativa,Não,Tarja -(*)


In [5]:
anterior = anterior.astype(str)
reajuste = reajuste.astype(str)

In [6]:
anterior.rename(columns = lambda x: x.replace(' ', '_'), inplace=True)
reajuste.rename(columns = lambda x: x.replace(' ', '_'), inplace=True)

In [7]:
anterior.replace({',': '.'}, regex=True, inplace=True)
reajuste.replace({',': '.'}, regex=True, inplace=True)

**Temos aqui as bases das últimas atualizações de medicamento no país, para inicia nossa EDA vamos verificar de quanto foi o aumento dos médicamentos.**

## Dicionário das variáveis 

**ANÁLISE RECURSAL** – Destina-se a prestar informações sobre produtos cuja análise de
preço ainda esteja em curso no âmbito da CMED, tanto em sede de pedido de reconsideração
como de recurso ao Comitê Técnico Executivo - CTE/CMED.

**APRESENTAÇÃO** – É a descrição de como o medicamento é apresentado, quanto a sua
forma farmacêutica, dosagem e quantidade.

**CAP** – O Coeficiente de Adequação de Preços é um desconto mínimo obrigatório,
atualizado anualmente, a ser aplicado sempre que forem realizadas vendas de medicamentos,
constantes do rol divulgado pela CMED e nas compras de todos os medicamentos por força de
decisão judicial, destinadas aos entes da administração pública direta e indireta da União, dos
Estados, do Distrito Federal e dos Municípios. Sobre ele, vamos falar mais à frente.

**CLASSE TERAPÊUTICA** – É a Classificação Anatômica de Produtos Farmacêuticos. Está
relacionado com a finalidade e o local de atuação do medicamento de acordo com a Associação
Europeia de Pesquisa de Mercado Farmacêutico (EphMRA).

**CNPJ** – Código de identificação da pessoa jurídica. 

**CÓDIGO GGREM** – É um campo específico de identificação de determinada
apresentação, cujo fornecimento e geração são exclusivos da CMED e são únicos para cada
apresentação.

**COMERCIALIZAÇÃO 2017** – Refere-se à informação se o produto foi ou não
comercializado pelo detentor do registro, no ano de 2017.

**CONFAZ 87** – Lista de medicamentos que ficam isentas do ICMS, cujas operações sejam
destinadas a órgãos da Administração Pública Direta e Indireta Federal, Estadual e Municipal e
do Distrito Federal e a suas fundações públicas.

**EAN 1** – O Código EAN é também conhecido como Código de Barras ou GTIN. É composto
por 13 dígitos e nele se identifica o País de origem do produto, a empresa fabricante, o produto
por ela produzido e um digito verificador.

**EAN 2** – O Código EAN é também conhecido como Código de Barras ou GTIN. É composto
por 13 dígitos e nele se identifica o País de origem do produto, a empresa fabricante, o produto
por ela produzido e um digito verificador.

**EAN 3** – O Código EAN é também conhecido como Código de Barras ou GTIN. É composto
por 13 dígitos e nele se identifica o País de origem do produto, a empresa fabricante, o produto
por ela produzido e um digito verificador.

**LABORATÓRIO** – É o nome da empresa detentora do registro sanitário podendo ser a
produtora ou importadora do medicamento.

**LISTA DE CONCESSÃO DE CRÉDITO TRIBUTÁRIO (PIS/COFINS)** – Classificação fiscal dos
medicamentos em uma das listas a seguir, conforme o regime tributário previsto na Lei nº
10.147, de 2000:

- **Lista Positiva:** medicamentos cujas substâncias ativas constam do anexo do Decreto nº 3.803, de 24 de abril de 2001, e suas atualizações, sujeitos à prescrição médica, identificados com tarja vermelha ou preta, e cujas empresas produtoras usufruem do regime especial de utilização de crédito presumido de PIS/Pasep e COFINS, classificados em três categorias, ou seja, medicamentos isentos de PIS/Pasep e COFINS:
    - I.Categoria I - Medicamentos monodroga identificados com tarja vermelha ou preta; 
    - II.Categoria II - Medicamentos em associações identificados com tarja vermelha ou preta;
    - III.Categoria III - Substâncias para medicamentos utilizados em nutrição parenteral, hemodiálise e diálise peritoneal, substitutos do plasma e       expansores plasmáticos, identificados com tarja vermelha. 
- **Lista Negativa:** medicamentos pertencentes às classificações constantes do art. 1º daLei nº 10.147, de 2000, cujas substâncias ativas não estejam relacionadas no anexo do Decreto nº 3.803, de 2001, e suas atualizações, ou seja, medicamentos incidência de 12% de PIS/Pasep e COFINS.
- **Lista Neutra:** medicamentos que não estão sujeitos ao regime tributário estabelecidona Lei nº 10.147, de 2000, ou seja, medicamentos incidência de 9,25% de PIS/Pasep e COFINS.

**PF** - PF 0%, PF 12%, PF 17%, PF 17% ALC, PF 17,5%, PF 17,5% ALC, PF 18%, PF 18% ALC, PF
20%, PF Sem Impostos, Preço Fábrica – É o teto de preço pelo qual um laboratório ou
distribuidor de medicamentos pode comercializar no mercado brasileiro um medicamento.
Desta maneira, o PF vem a ser o preço máximo permitido para vendas de medicamentos
destinadas a farmácias, drogarias, além das destinadas a entes da Administração Pública.

**PMC** - PMC 0%, PMC 12%, PMC 17%, PMC 17% ALC, PMC 17,5%, PMC 17,5% ALC, PMC 18%,
PMC 18% ALC, PMC 20%, Preço Máximo ao Consumidor ou PMC – É o preço a ser praticado pelo
comércio varejista de medicamentos, ou seja, farmácias e drogarias, tendo em vista que este
contempla tanto a margem de lucro como os impostos inerentes a esses tipos de comércio.

*A Alíquota de 0% se refere ao preço sem ICMS, que deve ser aplicado aos medicamentos isentos
de ICMS, que são aqueles relacionados nos Convênios do CONFAZ, por exemplo CONVÊNIO ICMS
162/94 que isenta medicamentos destinados ao tratamento de câncer.*

*A Alíquota de ICMS 20% se refere atualmente apenas ao Estado do RJ.*

*A Alíquota de ICMS 18% se refere aos Estados de AM, AP, BA, CE, MA, MG, PB, PE, PI, PR, RN,
RS, SE, SP, TO.*

*A Alíquota de ICMS 17,5% se refere ao Estado de RO*

*A alíquota de ICMS 17% está relacionada aos demais Estados*

*A Alíquota de ICMS 12% está relacionada com os Medicamentos Genéricos de SP e MG.*

*As Alíquotas de 17%, 17,5% e 18% que tem ao lado a sigla ALC, significa Áreas de Livre Comércio,
e dizem respeito às seguintes cidades: Manaus/Tabatinga (AM), Boa Vista/Bonfim (RR),
Macapá/Santana (AP), Guajará-Mirim (RO), Brasiléia/Epitaciolândia/ Cruzeiro do Sul (AC).*

*Além das alíquotas citadas anteriormente também é publicado o PF Sem Impostos que são os
preços sem o ICMS e sem PIS/Pasep e COFINS.*

**PRINCÍPIO ATIVO** – É uma substância química ativa, fármaco, droga ou matéria-prima
que tenha propriedades farmacológicas com finalidade medicamentosa, utilizada para
diagnóstico, alívio ou tratamento, empregada para modificar ou explorar sistemas fisiológicos
ou estados patológicos, em benefício da pessoa na qual se administra. Portanto, princípio ativo
do medicamento é o nome principal de sua composição.

**PRODUTO** – É o nome comercial dado ao medicamento, em determinada apresentação,
de um referido laboratório, no caso dos medicamentos genéricos é igual ao Princípio Ativo.

**REGISTRO** – É o número de registro de produto junto à ANVISA.

**RESTRIÇÃO HOSPITALAR** – Indica os medicamentos em embalagens hospitalares e de
uso restrito a hospitais e clínicas e, portanto, não podem ser comercializados pelo Preço Máximo
ao Consumidor não tendo seus PMC publicados.

**TARJA** – Indica qual se a venda do medicamento necessita de recita, sendo classificado em: 

- **Tarja Vermelha:** Medicamentos que só podem ser dispensados mediante apresentação de receita.
- **Tarja Preta:** Medicamentos que devem ser dispensados apenas mediante receituário acompanhado de notificação de receita, devendo ficar uma cópia no estabelecimento que realizou a dispensação.
- **Venda livre:** Medicamentos que podem ser dispensados sem a apresentação de receita.

**TIPO DE PRODUTO (STATUS DO PRODUTO)** – Tipo é a categorização do medicamento
por tipo de produto em Biológicos, Biológico Novo, Similar, Genérico, Novo, Específico,
Radiofármaco.

Status do produto indica se ele é referência.

**ICMS 0%** – Indica se o medicamento tem ou não o imposto sobre Circulação de
Mercadorias e Prestação de Serviços (ICMS), onde o imposto de cada Estado deve ser
considerado, pois ele influencia na diferença dos preços dos medicamentos de um Estado para
o outro. Por exemplo, no Estado do Paraná o ICMS é de 18% (dezoito por cento), no Estado do
Rio de Janeiro é de 20% (vinte por cento) e no Estado do Goiás é de 17% (dezessete por cento).

**Vamos unir os dataframes para comparação.**

In [8]:
df_merge = reajuste.merge(anterior, how = 'left', left_on = ['CNPJ_(REAJUSTE)', 'LABORATORIO_(REAJUSTE)', 'PRODUTO_(REAJUSTE)', 'APRESENTACAO_(REAJUSTE)', 'TARJA_(REAJUSTE)'], right_on = ['CNPJ', 'LABORATORIO', 'PRODUTO', 'APRESENTACAO', 'TARJA'])

In [9]:
df_merge['PF_Sem_Impostos_(REAJUSTE)'] = df_merge['PF_Sem_Impostos_(REAJUSTE)'].astype('float')
df_merge['PF_Sem_Impostos'] = df_merge['PF_Sem_Impostos'].astype('float')

In [10]:
df_merge

,SUBSTANCIA_(REAJUSTE),CNPJ_(REAJUSTE),LABORATORIO_(REAJUSTE),CODIGO_GGREM_(REAJUSTE),REGISTRO_(REAJUSTE),EAN_1_(REAJUSTE),EAN_2_(REAJUSTE),EAN_3_(REAJUSTE),PRODUTO_(REAJUSTE),APRESENTACAO_(REAJUSTE),CLASSE_TERAPEUTICA_(REAJUSTE),TIPO_DE_PRODUTO_(STATUS_DO_PRODUTO)_(REAJUSTE),REGIME_DE_PRECO_(REAJUSTE),PF_Sem_Impostos_(REAJUSTE),PF_0%_(REAJUSTE),PF_12%_(REAJUSTE),PF_17%_(REAJUSTE),PF_17%_ALC_(REAJUSTE),"PF_17,5%_(REAJUSTE)","PF_17,5%_ALC_(REAJUSTE)",PF_18%_(REAJUSTE),PF_18%_ALC_(REAJUSTE),PF_20%_(REAJUSTE),PMC_0%_(REAJUSTE),PMC_12%_(REAJUSTE),PMC_17%_(REAJUSTE),PMC_17%_ALC_(REAJUSTE),"PMC_17,5%_(REAJUSTE)","PMC_17,5%_ALC_(REAJUSTE)",PMC_18%_(REAJUSTE),PMC_18%_ALC_(REAJUSTE),PMC_20%_(REAJUSTE),RESTRICAO_HOSPITALAR_(REAJUSTE),CAP_(REAJUSTE),CONFAZ_87_(REAJUSTE),ICMS_0%_(REAJUSTE),ANALISE_RECURSAL_(REAJUSTE),LISTA_DE_CONCESSAO_DE_CREDITO_TRIBUTARIO_(PIS/COFINS)_(REAJUSTE),COMERCIALIZACAO_2019_(REAJUSTE),TARJA_(REAJUSTE),SUBSTANCIA,CNPJ,LABORATORIO,CODIGO_GGREM,REGISTRO,EAN_1,EAN_2,EAN_3,PRODUTO,APRESENTACAO,CLASSE_TERAPEUTICA,TIPO_DE_PRODUTO_(STATUS_DO_PRODUTO),REGIME_DE_PRECO,PF_Sem_Impostos,PF_0%,PF_12%,PF_17%,PF_17%_ALC,"PF_17,5%","PF_17,5%_ALC",PF_18%,PF_18%_ALC,PF_20%,PMC_0%,PMC_12%,PMC_17%,PMC_17%_ALC,"PMC_17,5%","PMC_17,5%_ALC",PMC_18%,PMC_18%_ALC,PMC_20%,RESTRICAO_HOSPITALAR,CAP,CONFAZ_87,ICMS_0%,ANALISE_RECURSAL,LISTA_DE_CONCESSAO_DE_CREDITO_TRIBUTARIO_(PIS/COFINS),COMERCIALIZACAO_2019,TARJA
0,ACETATO DE LANREOTIDA,07.718.721/0001-80,BEAUFOUR IPSEN FARMACÊUTICA LTDA,5.39E+14,1.70E+12,7.90E+12,7.90E+12,-,SOMATULINE AUTOGEL,120 MG SOL INJ LIB PROL CT ENV AL/PLAS SER PRE...,H1C2 - HORMÔNIOS ANTICRESCIMENTO,Novo,Regulado,2456.35,2456.35,2791.31,2959.46,2959.46,2977.39,2977.39,2995.55,2995.55,3070.44,3395.76,3858.82,4091.28,4091.28,4116.07,4116.07,4141.17,4141.17,4244.7,Não,Sim,Sim,Não,,Positiva,Não,Tarja -(*),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ACICLOVIR,45.992.062/0001-65,GERMED FARMACEUTICA LTDA,5.26E+14,1.06E+12,7.90E+12,-,-,ACICLOVIR,400 MG COM CT BL AL PLAS PVDC OPC X 20,J5B3 - ANTIVIRAIS PARA HERPES,Genérico,Regulado,57.69,57.69,65.56,69.51,69.51,69.93,69.93,70.35,70.35,72.11,79.75,90.63,96.09,96.09,96.67,96.67,97.25,97.25,99.69,Não,Não,Não,Não,,Positiva,Não,Tarja Vermelha(*),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ACICLOVIR,45.992.062/0001-65,GERMED FARMACEUTICA LTDA,5.26E+14,1.06E+12,7.90E+12,-,-,ACICLOVIR,400 MG COM CT BL AL PLAS PVDC OPC X 30,J5B3 - ANTIVIRAIS PARA HERPES,Genérico,Regulado,86.54,86.54,98.34,104.27,104.27,104.9,104.9,105.54,105.54,108.18,119.64,135.95,144.15,144.15,145.02,145.02,145.9,145.9,149.55,Não,Não,Não,Não,,Positiva,Não,Tarja Vermelha(*),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ACICLOVIR,45.992.062/0001-65,GERMED FARMACEUTICA LTDA,5.26E+14,1.06E+12,7.90E+12,-,-,ACICLOVIR,400 MG COM CT BL AL PLAS PVDC OPC X 50,J5B3 - ANTIVIRAIS PARA HERPES,Genérico,Regulado,144.23,144.23,163.9,173.77,173.77,174.82,174.82,175.89,175.89,180.29,199.39,226.58,240.23,240.23,241.68,241.68,243.16,243.16,249.24,Não,Não,Não,Não,,Positiva,Não,Tarja Vermelha(*),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACTAEA RACEMOSA L.,78.950.011/0001-20,HERBARIUM LABORATORIO BOTANICO LTDA,5.11E+14,1.19E+12,7.90E+12,-,-,CLIFEMIN,160 MG COM REV CT BL AL PLAS PVDC TRANS X 15,G3J - MODULADORES SELETIVOS DO RECEPTOR DE EST...,Fitoterápico,Liberado,34.10,38.19,44.13,47.19,41.08,47.52,41.33,47.86,41.58,49.24,51.23,58.95,62.9,56.79,63.33,57.14,63.76,57.48,65.54,Não,Não,Não,Não,,Negativa,Não,Tarja -(*),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

**Agora vamos fazer uma comparação entre o valor anterior com o reajuste**

In [11]:
for index, row in df_merge.iterrows():
    if row['APRESENTACAO_(REAJUSTE)'] == row['APRESENTACAO'] and row['PRODUTO_(REAJUSTE)'] == row['PRODUTO'] and row['LABORATORIO_(REAJUSTE)'] == row['LABORATORIO']:
        df_merge.loc[index,'VALOR_REAJUSTE_%'] =  100*((row['PF_Sem_Impostos_(REAJUSTE)']/row['PF_Sem_Impostos'])- 1)

In [12]:
verificacao = df_merge[['LABORATORIO_(REAJUSTE)', 'PRODUTO_(REAJUSTE)', 'APRESENTACAO_(REAJUSTE)', 'LABORATORIO', 'PRODUTO', 'APRESENTACAO', 'PF_Sem_Impostos', 'PF_Sem_Impostos_(REAJUSTE)', 'VALOR_REAJUSTE_%']]
verificacao

,LABORATORIO_(REAJUSTE),PRODUTO_(REAJUSTE),APRESENTACAO_(REAJUSTE),LABORATORIO,PRODUTO,APRESENTACAO,PF_Sem_Impostos,PF_Sem_Impostos_(REAJUSTE),VALOR_REAJUSTE_%
0,BEAUFOUR IPSEN FARMACÊUTICA LTDA,SOMATULINE AUTOGEL,120 MG SOL INJ LIB PROL CT ENV AL/PLAS SER PRE...,NaN,NaN,NaN,NaN,2456.35,NaN
1,GERMED FARMACEUTICA LTDA,ACICLOVIR,400 MG COM CT BL AL PLAS PVDC OPC X 20,NaN,NaN,NaN,NaN,57.69,NaN
2,GERMED FARMACEUTICA LTDA,ACICLOVIR,400 MG COM CT BL AL PLAS PVDC OPC X 30,NaN,NaN,NaN,NaN,86.54,NaN
3,GERMED FARMACEUTICA LTDA,ACICLOVIR,400 MG COM CT BL AL PLAS PVDC OPC X 50,NaN,NaN,NaN,NaN,144.23,NaN
4,HERBARIUM LABORATORIO BOTANICO LTDA,CLIFEMIN,160 MG COM REV CT BL AL PLAS PVDC TRANS X 15,NaN,NaN,NaN,NaN,34.10,NaN
...,...,...,...,...,...,...,...,...,...
26002,SANOFI-AVENTIS FARMACÊUTICA LTDA,IMOVANE,7.5 MG COM REV CT BL AL PLAS TRANS X 20,NaN,NaN,NaN,NaN,35.36,NaN
26003,LUNDBECK BRASIL LTDA,CLOPIXOL,10 MG COM REV CT BL AL PLAS TRANS X 20,NaN,NaN,NaN,NaN,26.08,NaN
26004,LUNDBECK BRASIL LTDA,CLOPIXOL,25 MG COM REV CT BL AL PLAS TRANS X 20,LUNDBECK BRASIL LTDA,CLOPIXOL,25 MG COM REV CT BL AL PLAS TRANS X 20,57.02,57.02,0.0
26005,LUNDBECK BRASIL LTDA,CLOPIXOL,50 MG/ML SOL INJ ACUPHASE CT AMP VD TRANS X 1 ML,NaN,NaN,NaN,NaN,26.22,NaN


In [13]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [14]:
verificacao['VALOR_REAJUSTE_%'].value_counts()

0.00      999
-4.05       2
4.22        2
5.24        2
4.22        2
5.17        1
4.22        1
5.19        1
-4.99       1
3.27        1
3.16        1
4.23        1
4.21        1
2.99        1
3.21        1
4.08        1
3.16        1
37.79       1
5.21        1
4.21        1
-4.05       1
5.20        1
3.23        1
5.21        1
5.21        1
3.22        1
5.20        1
4.21        1
4.27        1
4.23        1
65.99       1
-12.46      1
-4.96       1
3.22        1
5.21        1
3.22        1
5.21        1
4.13        1
5.26        1
4.22        1
4.21        1
Name: VALOR_REAJUSTE_%, dtype: int64

**Temos aqui os valores em porcentagem do reajuste, tivemos 999 medicamentos que não tiveram reajuste em seu preço.**

**Agora vamos verificar quais medicamentos tiveram aumento em seus preços olhado para o seu preço base, ou seja, sem impostos.**

In [15]:
aumento = verificacao[verificacao['VALOR_REAJUSTE_%'] > 0]

In [16]:
maior_aumento = aumento.sort_values(by='VALOR_REAJUSTE_%', ascending=False)
maior_aumento.head(25)

,LABORATORIO_(REAJUSTE),PRODUTO_(REAJUSTE),APRESENTACAO_(REAJUSTE),LABORATORIO,PRODUTO,APRESENTACAO,PF_Sem_Impostos,PF_Sem_Impostos_(REAJUSTE),VALOR_REAJUSTE_%
12880,BESINS HEALTHCARE BRASIL COMERCIAL E DISTRIBUI...,VAGIFEM,10 MCG COM REV VAG CT ENVOL APLIC PREENC PLAS ...,BESINS HEALTHCARE BRASIL COMERCIAL E DISTRIBUI...,VAGIFEM,10 MCG COM REV VAG CT ENVOL APLIC PREENC PLAS ...,40.49,67.21,65.99
25399,EUROFARMA LABORATÓRIOS S.A.,SUBTRAX,80 MG/ML + 16 MG/ML SUS OR CT FR VD AMB X 100 ML,EUROFARMA LABORATÓRIOS S.A.,SUBTRAX,80 MG/ML + 16 MG/ML SUS OR CT FR VD AMB X 100 ML,17.20,23.70,37.79
17496,EMS S/A,LIMECICLINA,300 MG CAP DURA CT BL AL AL X 16,EMS S/A,LIMECICLINA,300 MG CAP DURA CT BL AL AL X 16,59.93,63.08,5.26
22118,LABORATÓRIOS PFIZER LTDA,ALOND,25 MG CAP DURA CT BL AL PLAS PVC TRANS X 10,LABORATÓRIOS PFIZER LTDA,ALOND,25 MG CAP DURA CT BL AL PLAS PVC TRANS X 10,7.64,8.04,5.24
22111,APSEN FARMACEUTICA S/A,INSIT®,25 MG CAP DURA CT BL AL PLAS PVC TRANS X 15,APSEN FARMACEUTICA S/A,INSIT®,25 MG CAP DURA CT BL AL PLAS PVC TRANS X 15,11.46,12.06,5.24
22117,LABORATÓRIOS PFIZER LTDA,ALOND,150 MG CAP DURA CT BL AL PLAS PVC TRANS X 28,LABORATÓRIOS PFIZER LTDA,ALOND,150 MG CAP DURA CT BL AL PLAS PVC TRANS X 28,125.82,132.38,5.21
20145,EMS S/A,ZOPIX,5 MG COM REV CT BL AL AL X 28,EMS S/A,ZOPIX,5 MG COM REV CT BL AL AL X 28,220.06,231.53,5.21
2074,EMS S/A,ARPEJO,20 MG/ML SUS GOT OR CT ENVOL FR GOT PLAS PEAD/...,EMS S/A,ARPEJO,20 MG/ML SUS GOT OR CT ENVOL FR GOT PLAS PEAD/...,797.14,838.67,5.21
20144,EMS S/A,ZOPIX,2.5 MG COM REV CT BL AL AL X 28,EMS S/A,ZOPIX,2.5 MG COM REV CT BL AL AL X 28,148.95,156.71,5.21
22116,LABORATÓRIOS PFIZER LTDA,ALOND,75 MG CAP DURA CT BL AL PLAS PVC TRANS X 28,LABORATÓRIOS PFIZER LTDA,ALOND,75 MG CAP DURA CT BL AL PLAS PVC TRANS X 28,74.31,78.18,5.21


**Os dois medicamentos com maior aumnto foram o VAGIFEM 10 MCG COM REV VAG CT ENVOL APLIC PREENC PLAS eo SUBTRAX 80 MG/ML + 16 MG/ML SUS OR CT FR VD AMB X 100 ML os demais medicamentos ficaram com seu reajuste abaixo de 6%**

**Agora iremos verificar quais medicamentos tiveram queda em seus preços.**